# Atomic multiplets
Two different (n,l)-shells are considered, where the first shell is initially considered to be fully occupied.
All possible occupations for the second shell is considered. 

The spin-orbitals are ordered: 
$b_{1,1,\downarrow}, b_{1,1,\uparrow}, b_{1,2,\downarrow}, b_{1,2,\uparrow}, b_{1,3,\downarrow}, b_{1,3,\uparrow},  ..., b_{1,nb_1/2,\downarrow}, b_{1,nb_1/2,\uparrow},
b_{2,1,\downarrow}, b_{2,1,\uparrow}, b_{2,2,\downarrow}, b_{2,2,\uparrow}, b_{2,3,\downarrow}, b_{2,3,\uparrow},  ..., b_{2,nb_2/2,\downarrow}, b_{2,nb_2/2,\uparrow}$.

where the first index indicates shell, the second index the $l_z$, and the last index the spin. 


In [16]:
import numpy as np
import itertools


def hundsGroundState(nb, n):
    '''
    Return L, S and J for the ground state according to Hunds rule.
    
    Parameters
    ----------
    nb : int
        Number of spin-orbitals
    n : int
        Number of electrons
    '''
    S,L,nTot = 0,0,n
    for sz in [1/2, -1/2]:
        for lz in range((nb-2)//4,-(nb-2)//4-1,-1):
            if n > 0:
                S += sz
                L += lz
                n -= 1
            if n == 0:
                J = L+S if nb/2 < nTot else abs(L-S) 
                return (L,S,J)

def getSz(i):
    return -1/2 if i%2 == 0 else 1/2

def getLz(i,nb1,nb2):
    return i//2 - (nb1-2)/4 if i<nb1 else (i-nb1)//2 - (nb2-2)/4

def findMultiplets(nb1,n1,nb2,n2):
    '''
    Return all multiplets given the occupation in two shells.
    
    Parameters
    ----------
    nb1 : int
        Number of spin-orbitals in the first shell
    n1 : int
        Number of electrons in the first shell
    nb2 : int
        Number of spin-orbitals in the second shell
    n2 : int
        Number of electrons in the second shell
        
    '''
    # Check validity of input data
    if not (nb1%2 == 0 and (nb1//2-1)%2 == 0): 
        raise ValueError('nb1 needs to fulfill: nb1 = 2*(2*L+1), L non-negative integer') 
    if not (nb2%2 == 0 and (nb2//2-1)%2 == 0): 
        raise ValueError('nb2 needs to fulfill: nb2 = 2*(2*L+1), L non-negative integer') 

    # Create basis.
    # For each shell, create all configurations
    # given the occupation in that shell.
    states1 = tuple(itertools.combinations(range(nb1),n1))
    states2 = tuple(itertools.combinations(nb1+np.arange(nb2),n2))
    states = []
    for s1 in states1:
        for s2 in states2:
            states.append(s1+s2)
    states = tuple(states)
    #print 'Number of states:',len(states)
    # Measure Lz and Sz for each state
    lzSz = {}
    for s in states:
        lz = 0
        sz = 0
        for i in s:
            lz += getLz(i,nb1,nb2)
            sz += getSz(i)
        if (lz,sz) in lzSz:
            lzSz[(lz,sz)] += 1
        else:
            lzSz[(lz,sz)] = 1
    # Calculate L and S
    ls = []
    while lzSz:
        # Find Lz max
        lmax = [] 
        for c in lzSz:
            lmax.append(c[0])
        lmax = max(lmax)
        # Given Lz max, find Sz max
        smax = []
        for c in lzSz:
            if c[0] == lmax:
                smax.append(c[1])
        smax = max(smax)
        # Save found L and S
        ls.append((lmax,smax))
        # Remove one configuration from each (Lz,Sz) pair fullfilling
        # |Lz|<=LzMax and |Sz|<=SzMax
        keys = []
        for c in lzSz:
            if abs(c[0]) <= lmax and abs(c[1]) <= smax:
                if lzSz[c] == 1:
                    keys.append(c)
                else:
                    lzSz[c] -= 1
        for key in keys:
            lzSz.pop(key)
            
    multiplets = []
    for (L,S) in ls:
        for j in np.arange(abs(L-S),L+S+1):
            multiplets.append((L,S,j))
    return multiplets

In [17]:
# Number of spin-orbitals in first and second shell
nb1, nb2 = 6, 10
print('    Transition     GroundState(L,S,J) #Transitions #TermSymbols')
for n2 in range(nb2):    
    lsj0 = hundsGroundState(nb2, n2)     
    multiplets = findMultiplets(nb1, nb1-1, nb2, n2+1)
    transitions = []
    for m in multiplets:
        # Dipole selection rules: DeltaJ = -1,1 or DeltaJ=0 if J>0
        if abs(m[2]-lsj0[2])==1 or (m[2]==lsj0[2] and m[2]>0):
            transitions.append(m)
    print('3d^{:d} -> (2p^5 3d^{:d})  ({:d},{:3.1f},{:3.1f}) ' 
           '{:12d} {:12d}'.format(n2,n2+1,lsj0[0],lsj0[1],lsj0[2],
                                   len(transitions),len(multiplets))) 

    Transition     GroundState(L,S,J) #Transitions #TermSymbols
3d^0 -> (2p^5 3d^1)  (0,0.0,0.0)            3           12
3d^1 -> (2p^5 3d^2)  (2,0.5,1.5)           29           45
3d^2 -> (2p^5 3d^3)  (3,1.0,2.0)           68          110
3d^3 -> (2p^5 3d^4)  (3,1.5,1.5)           95          180
3d^4 -> (2p^5 3d^5)  (2,2.0,0.0)           32          214
3d^5 -> (2p^5 3d^6)  (0,2.5,2.5)          110          180
3d^6 -> (2p^5 3d^7)  (2,2.0,4.0)           53          110
3d^7 -> (2p^5 3d^8)  (3,1.5,4.5)           16           45
3d^8 -> (2p^5 3d^9)  (3,1.0,4.0)            4           12
3d^9 -> (2p^5 3d^10)  (2,0.5,2.5)            1            2
